In [9]:
!python --version

Python 3.11.0rc1


In [10]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


In [11]:
!nvidia-smi

Wed Jun 12 07:26:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:65:00.0 Off |                  N/A |
|  0%   34C    P8              21W / 300W |   9989MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!python -m pip freeze > requirements.txt

### Load the dataset

In [1]:
import pandas as pd
dataset = pd.read_csv('datasets/covid19-twitter.tsv', sep="\t")
dataset

,tweet_id,emojis,emoji_names,emotion_scores,agg_emotion_score,sentiment_label,vader_score_sentence,vader_score_clause_A,vader_score_clause_B,vader_sentiment_sentence,vader_sentiment_clause_A,vader_sentiment_clause_B,rule_structure,rule_label,contrast
0,1292641373758328839,"['😢', '😢']","['crying face', 'crying face']","[[-0.25, -0.36, -0.5, 0.0, -1.0, 0.11], [-0.25...",-4.00,negative,"{'neg': 0.249, 'neu': 0.751, 'pos': 0.0, 'comp...",not_applicable,not_applicable,negative,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
1,1377497788460720129,"['😂', '😂', '😂', '😂']","['face with tears of joy', 'face with tears of...","[[0.0, -0.06, -0.06, 0.94, 0.0, 0.22], [0.0, -...",4.16,positive,"{'neg': 0.021, 'neu': 0.909, 'pos': 0.07, 'com...","{'neg': 0.036, 'neu': 0.909, 'pos': 0.056, 'co...","{'neg': 0.0, 'neu': 0.9, 'pos': 0.1, 'compound...",positive,positive,positive,A-but-B,A-but-B,no_contrast
2,1318434591322087427,"['💔', '💔', '💔']","['broken heart', 'broken heart', 'broken heart']","[[-0.39, -0.33, -0.14, 0.0, -0.94, 0.17], [-0....",-4.89,negative,"{'neg': 0.176, 'neu': 0.824, 'pos': 0.0, 'comp...",not_applicable,not_applicable,negative,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
3,1303627865796222976,"['😠', '😠', '😠', '😠', '😠']","['angry face', 'angry face', 'angry face', 'an...","[[-1.0, -0.56, -0.17, 0.0, -0.25, 0.08], [-1.0...",-9.50,negative,"{'neg': 0.117, 'neu': 0.883, 'pos': 0.0, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.302, 'neu': 0.698, 'pos': 0.0, 'comp...",negative,neutral,negative,A-yet-B,A-yet-B,contrast
4,1361789228426932227,"['🎉', '🎉', '🎉']","['party popper', 'party popper', 'party popper']","[[0.0, 0.0, 0.0, 0.92, 0.0, 0.33], [0.0, 0.0, ...",3.75,positive,"{'neg': 0.0, 'neu': 0.686, 'pos': 0.314, 'comp...",not_applicable,not_applicable,positive,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109914,1304993476564590592,"['😡', '😡', '😡']","['pouting face', 'pouting face', 'pouting face']","[[-1.0, -0.56, -0.11, 0.0, -0.36, 0.06], [-1.0...",-5.91,negative,"{'neg': 0.192, 'neu': 0.808, 'pos': 0.0, 'comp...",not_applicable,not_applicable,negative,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
109915,1389249469850394627,"['💐', '💐', '💐', '💐', '👍']","['bouquet', 'bouquet', 'bouquet', 'bouquet', '...","[[0.0, 0.0, 0.0, 0.69, -0.11, 0.58], [0.0, 0.0...",5.19,positive,"{'neg': 0.0, 'neu': 0.291, 'pos': 0.709, 'comp...",not_applicable,not_applicable,positive,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
109916,1322678255128903683,"['😢', '😢']","['crying face', 'crying face']","[[-0.25, -0.36, -0.5, 0.0, -1.0, 0.11], [-0.25...",-4.00,negative,"{'neg': 0.106, 'neu': 0.846, 'pos': 0.049, 'co...","{'neg': 0.0, 'neu': 0.822, 'pos': 0.178, 'comp...","{'neg': 0.133, 'neu': 0.867, 'pos': 0.0, 'comp...",negative,positive,negative,A-but-B,A-but-B,contrast
109917,1361083777724915712,"['🙏', '🙏', '🙏', '🙏', '🙏', '🙏', '🙏', '🙏', '🙏', ...","['folded hands', 'folded hands', 'folded hands...","[[-0.06, 0.0, -0.11, 0.25, -0.11, 0.58], [-0.0...",5.50,positive,"{'neg': 0.0, 'neu': 0.79, 'pos': 0.21, 'compou...","{'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",positive,positive,neutral,A-while-B,A-while-B,contrast


### Hydrate tweets from tweet-IDs using tweepy

In [ ]:
import tweepy

bearer_token="YOUR_BEARER_TOKEN_VALUE"
api_key = 'YOUR_API_KEY_VALUE'
api_secret = 'YOUR_API_SECRET_VALUE'
access_token = 'YOUR_ACCESS_TOKEN_VALUE'
access_token_secret = 'YOUR_ACCESS_TOKEN_SECRET_VALUE'

client = tweepy.Client(bearer_token)
client = tweepy.Client(consumer_key=consumer_key, 
                       consumer_secret=consumer_secret,
                       access_token=access_token, 
                       access_token_secret=access_token_secret)

tweet_ids = list(dataset["tweet_id"])
response = client.get_tweets(tweet_ids)

tweets = []
for index, tweet in enumerate(response.data):
    assert(tweet.id==tweet_ids[index])
    tweets.append(tweet.text)
dataset["tweet"] = tweets

### Option 1: Reproduce benchmark results

#### Specifiy the parameters

In [37]:
#Identify the configuration parameters for every benchmark model
import pickle
import pprint

FLAT_CLASSIFIERS = [
                  "mlp_1e-6LearningRate_200Epochs_0.5Dropout",
                  "cnn_1e-6LearningRate_200Epochs_0.5Dropout",
                  "lstm_1e-6LearningRate_200Epochs_0.5Dropout",
                  "bilstm_1e-6LearningRate_200Epochs_0.5Dropout",
                  "gru_1e-6LearningRate_200Epochs_0.5Dropout",
                  "bigru_1e-6LearningRate_200Epochs_0.5Dropout",
                  "transformer_1e-6LearningRate_200Epochs_0.5Dropout_2Heads_LimeExpFixed"
                 ]

BASELINES = [
            "bertweet_mlp_DropoutAddedInMlp",
            "bertweet_cnn",
            "bertweet_lstm",
            "bertweet_bilstm",
            "bertweet_gru",
            "bertweet_bigru",
            "bertweet_transformer",
            "gpt2_mlp",
            "gpt2_cnn",
            "gpt2_lstm",
            "gpt2_bilstm",
            "gpt2_gru",
            "gpt2_bigru",
            "gpt2_transformer",
            "xlnet_mlp_20BatchSize",
            "xlnet_cnn_20BatchSize",
            "xlnet_lstm_20BatchSize",
            "xlnet_bilstm_20BatchSize",
            "xlnet_gru_20BatchSize",
            "xlnet_bigru_20BatchSize",
            "xlnet_transformer_20BatchSize",
            "roberta_mlp_20BatchSize",
            "roberta_cnn_20BatchSize",
            "roberta_lstm_20BatchSize",
            "roberta_bilstm_20BatchSize",
            "roberta_gru_20BatchSize",
            "roberta_bigru_20BatchSize",
            "roberta_transformer_20BatchSize",
            "distilbert_mlp_20BatchSize",
            "distilbert_cnn_20BatchSize",
            "distilbert_lstm_20BatchSize",
            "distilbert_bilstm_20BatchSize",
            "distilbert_gru_20BatchSize",
            "distilbert_bigru_20BatchSize",
            "distilbert_transformer_20BatchSize",
          ]

for index, classifier in enumerate(FLAT_CLASSIFIERS):
    with open("assets/configurations/"+classifier+".pickle", "rb") as handle:
        config = pickle.load(handle)
    pprint.pprint(config)
    print("\n")

{'asset_name': 'mlp_1e-6LearningRate_200Epochs_0.5Dropout',
 'dataset_name': 'covid19-twitter',
 'dataset_path': 'datasets/covid19-twitter/raw_dataset.pickle',
 'dropout': 0.5,
 'evaluate_model': True,
 'fine_tune_word_embeddings': False,
 'generate_explanation': True,
 'generate_explanation_for_one_instance': False,
 'hidden_units': 128,
 'learning_rate': 1e-06,
 'lime_no_of_samples': 1000,
 'mini_batch_size': 50,
 'model_name': 'mlp',
 'optimizer': 'adam',
 'seed_value': 11,
 'train_epochs': 200,
 'train_model': True}


{'asset_name': 'cnn_1e-6LearningRate_200Epochs_0.5Dropout',
 'dataset_name': 'covid19-twitter',
 'dataset_path': 'datasets/covid19-twitter/raw_dataset.pickle',
 'dropout': 0.5,
 'evaluate_model': True,
 'fine_tune_word_embeddings': False,
 'generate_explanation': True,
 'generate_explanation_for_one_instance': False,
 'hidden_units': 100,
 'learning_rate': 1e-06,
 'lime_no_of_samples': 1000,
 'mini_batch_size': 50,
 'model_name': 'cnn',
 'optimizer': 'adam',
 'seed_va

#### Run the models

In [ ]:
#Specify of configuration parameters of each benchmark model whose result need to reproduced 
##Run the previous code to identify the parameters of the desired model
##If you would just like to check the existing trained models, set the "train_model" value to False
config = {
            "asset_name":"bertweet_gru_reproducibility_test",
            "model_name":"bertweet_gru",
            "seed_value":11,
            "dataset_name":"covid19-twitter",
            "train_model":False,
            "evaluate_model":False,
            "generate_explanations":False,
            "generate_explanation_for_one_instance":True,
            "fine_tune_word_embeddings":True,
            "optimizer":"adam",
            "learning_rate":1e-6,
            "mini_batch_size":50,
            "train_epochs":2,
            "dropout":0.5,
            "lime_no_of_samples":1000,
            "hidden_units":128
        }

In [3]:
#import packages
import argparse
import pprint
import os
import logging
import warnings
import random
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
print("\nNum GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import argparse
import subprocess as sp
import distutils
import pprint

#change the code execution directory to current directory
os.chdir(os.getcwd())

#scripts
from scripts.preprocess_dataset import Preprocess_dataset
from scripts.word_vectors import Word_vectors
from scripts.dataset_division import Dataset_division

from scripts.train_mlp import train_mlp
from scripts.train_transformer import train_transformer
from scripts.train_cnn import train_cnn
from scripts.train_rnn import train_rnn

from scripts.train_bertweet_mlp import train_bertweet_mlp
from scripts.train_bertweet_transformer import train_bertweet_transformer
from scripts.train_bertweet_cnn import train_bertweet_cnn
from scripts.train_bertweet_rnn import train_bertweet_rnn
from scripts.train_gpt2_mlp import train_gpt2_mlp
from scripts.train_gpt2_transformer import train_gpt2_transformer
from scripts.train_gpt2_rnn import train_gpt2_rnn
from scripts.train_gpt2_cnn import train_gpt2_cnn
from scripts.train_roberta_mlp import train_roberta_mlp
from scripts.train_roberta_cnn import train_roberta_cnn
from scripts.train_roberta_transformer import train_roberta_transformer
from scripts.train_roberta_rnn import train_roberta_rnn
from scripts.train_xlnet_mlp import train_xlnet_mlp
from scripts.train_xlnet_cnn import train_xlnet_cnn
from scripts.train_xlnet_transformer import train_xlnet_transformer
from scripts.train_xlnet_rnn import train_xlnet_rnn
from scripts.train_distilbert_mlp import train_distilbert_mlp
from scripts.train_distilbert_cnn import train_distilbert_cnn
from scripts.train_distilbert_transformer import train_distilbert_transformer
from scripts.train_distilbert_rnn import train_distilbert_rnn

#disable warnings
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

#set the gpu execution device with highest free memory
def mask_unused_gpus(leave_unmasked=1): # No of avaialbe GPUs on the system
    COMMAND = "nvidia-smi --query-gpu=memory.free --format=csv"
    try:
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        memory_free_info = _output_to_list(sp.check_output(COMMAND.split()))[1:]
        memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
        available_gpus = [i for i, x in enumerate(memory_free_values)]
        if len(available_gpus) < leave_unmasked: raise ValueError('Found only %d usable GPUs in the system' % len(available_gpus))
        gpu_with_highest_free_memory = 0
        highest_free_memory = 0
        for index, memory in enumerate(memory_free_values):
            if memory > highest_free_memory:
                highest_free_memory = memory
                gpu_with_highest_free_memory = index
        return str(gpu_with_highest_free_memory)
    except Exception as e:
        print('"nvidia-smi" is probably not installed. GPUs are not masked', e)
os.environ["CUDA_VISIBLE_DEVICES"] = mask_unused_gpus()

#prepare the dataset
print("\nCreating input data")
preprocessed_dataset = Preprocess_dataset(config).preprocess_covid_tweets(dataset)
word_vectors, word_index = Word_vectors(config).create_word_vectors(preprocessed_dataset)
train_dataset, val_datasets, test_datasets = Dataset_division(config).train_val_test_split(preprocessed_dataset)

#create model
print("\nBuilding model")

#flat classifiers
if config["model_name"] == "mlp":
    train_mlp(config).train_model(train_dataset, val_datasets, test_datasets, word_index, word_vectors)
elif config["model_name"] == "transformer":
    train_transformer(config).train_model(train_dataset, val_datasets, test_datasets, word_index, word_vectors)
elif config["model_name"] == "cnn":
    train_cnn(config).train_model(train_dataset, val_datasets, test_datasets, word_index, word_vectors)
elif config["model_name"] in ["lstm", "bilstm", "gru", "bigru"]:
    train_rnn(config).train_model(train_dataset, val_datasets, test_datasets, word_index, word_vectors)

#cwe classifiers
elif config["model_name"] == "bertweet_mlp":
    train_bertweet_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "bertweet_transformer":
    train_bertweet_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "bertweet_cnn":
    train_bertweet_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["bertweet_lstm", "bertweet_bilstm", "bertweet_gru", "bertweet_bigru"]:
    train_bertweet_rnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "gpt2_mlp":
    train_gpt2_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "gpt2_transformer":
    train_gpt2_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["gpt2_lstm", "gpt2_bilstm", "gpt2_gru", "gpt2_bigru"]:
    train_gpt2_rnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "gpt2_cnn":
    train_gpt2_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "roberta_mlp":
    train_roberta_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "roberta_cnn":
    train_roberta_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "roberta_transformer":
    train_roberta_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["roberta_lstm", "roberta_bilstm", "roberta_gru", "roberta_bigru"]:
    train_roberta_rnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "xlnet_mlp":
    train_xlnet_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "xlnet_cnn":
    train_xlnet_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "xlnet_transformer":
    train_xlnet_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["xlnet_lstm", "xlnet_bilstm", "xlnet_gru", "xlnet_bigru"]:
    train_xlnet_rnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "distilbert_mlp":
    train_distilbert_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "distilbert_cnn":
    train_distilbert_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "distilbert_transformer":
    train_distilbert_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["distilbert_lstm", "distilbert_bilstm", "distilbert_gru", "distilbert_bigru"]:
    train_distilbert_rnn(config).train_model(train_dataset, val_datasets, test_datasets)

2024-06-13 12:00:54.472369: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 12:00:54.472405: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 12:00:54.472437: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 12:00:54.480405: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



Num GPUs Available:  1

Creating input data


2024-06-13 12:02:00.987743: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9689 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5



Word vectors created

Converted 36063 words (27654 misses)

Building model
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 81)]                 0         []                            
                                                                                                  
 tf.cast (TFOpLambda)        (None, 81)                   0         ['input_1[0][0]']             
                                                                                                  
 tf.math.not_equal (TFOpLam  (None, 81)                   0         ['tf.cast[0][0]']             
 bda)                                                                                             
                                                                                                  
 tf.cast_1 (TFOpLa

### Option 2: Check the existing results presented in the paper

In [1]:
import pickle
import pandas as pd
import os
from scripts.corrects_distribution import Corrects_distribution
from scripts.percy_score import calculate_percy

FLAT_CLASSIFIERS = [
                  "mlp_1e-6LearningRate_200Epochs_0.5Dropout",
                  "cnn_1e-6LearningRate_200Epochs_0.5Dropout",
                  "lstm_1e-6LearningRate_200Epochs_0.5Dropout",
                  "bilstm_1e-6LearningRate_200Epochs_0.5Dropout",
                  "gru_1e-6LearningRate_200Epochs_0.5Dropout",
                  "bigru_1e-6LearningRate_200Epochs_0.5Dropout",
                  "transformer_1e-6LearningRate_200Epochs_0.5Dropout_2Heads_LimeExpFixed"
                 ]

CWE = [
        "bertweet_mlp_DropoutAddedInMlp",
        "bertweet_cnn",
        "bertweet_lstm",
        "bertweet_bilstm",
        "bertweet_gru",
        "bertweet_bigru",
        "bertweet_transformer",
        "gpt2_mlp",
        "gpt2_cnn",
        "gpt2_lstm",
        "gpt2_bilstm",
        "gpt2_gru",
        "gpt2_bigru",
        "gpt2_transformer",
        "xlnet_mlp_20BatchSize",
        "xlnet_cnn_20BatchSize",
        "xlnet_lstm_20BatchSize",
        "xlnet_bilstm_20BatchSize",
        "xlnet_gru_20BatchSize",
        "xlnet_bigru_20BatchSize",
        "xlnet_transformer_20BatchSize",
        "roberta_mlp_20BatchSize",
        "roberta_cnn_20BatchSize",
        "roberta_lstm_20BatchSize",
        "roberta_bilstm_20BatchSize",
        "roberta_gru_20BatchSize",
        "roberta_bigru_20BatchSize",
        "roberta_transformer_20BatchSize",
        "distilbert_mlp_20BatchSize",
        "distilbert_cnn_20BatchSize",
        "distilbert_lstm_20BatchSize",
        "distilbert_bilstm_20BatchSize",
        "distilbert_gru_20BatchSize",
        "distilbert_bigru_20BatchSize",
        "distilbert_transformer_20BatchSize",
      ]

for index, classifier in enumerate(FLAT_CLASSIFIERS):
    with open("assets/configurations/"+classifier+".pickle", "rb") as handle:
        configuration = pickle.load(handle)
    model_name = configuration["model_name"]
    
    with open("assets/results/"+classifier+".pickle", "rb") as handle:
        results = pickle.load(handle)
        results = pd.DataFrame(results)
    one_rule = pd.concat([results.loc[(results["rule_label"]!=0)&(results["contrast"]==1)], results.loc[(results["rule_label"]!=0)&(results["contrast"]==0)]])
    one_rule = one_rule.reset_index(drop=True)
    one_rule_contrast_pos = one_rule.loc[(one_rule["contrast"]==1)&(one_rule["sentiment_label"]==1)]
    one_rule_contrast_neg = one_rule.loc[(one_rule["contrast"]==1)&(one_rule["sentiment_label"]==0)]
    one_rule_contrast_pos_sample = one_rule_contrast_pos.sample(n=1350, random_state=11)
    one_rule_contrast_neg_sample = one_rule_contrast_neg.sample(n=1351, random_state=11)
    one_rule.drop(one_rule_contrast_pos_sample.index, inplace = True)
    one_rule.drop(one_rule_contrast_neg_sample.index, inplace = True)
    results = pd.concat([results.loc[results["rule_label"]==0], one_rule])
    results = results.reset_index(drop=True)
    base_sent_corrects = Corrects_distribution(len(results['tweet_id'])).model_sentiment_correct_distributions(results)
    accuracy = sum(base_sent_corrects["one_rule"])/len(base_sent_corrects["one_rule"])
    print("\n")
    print(model_name+" sentiment accuracy: ", round(accuracy, 3))
    
    with open("assets/lime_explanations/"+classifier+".pickle", "rb") as handle:
        explanations = pickle.load(handle)
        explanations = pd.DataFrame(explanations)
    explanations.drop(one_rule_contrast_pos_sample.index, inplace = True)
    explanations.drop(one_rule_contrast_neg_sample.index, inplace = True)
    explanations = explanations.reset_index(drop=True)
    one_rule_results = pd.concat([results.loc[(results["rule_label"]!=0)&(results["contrast"]==1)], 
                                  results.loc[(results["rule_label"]!=0)&(results["contrast"]==0)]])
    percy = calculate_percy(one_rule_results, explanations)
    percy_value = sum(percy["one_rule"])/len(percy["one_rule"])
    print("\n")
    print(model_name+" PERCY score: ", round(percy_value, 3))

2024-06-13 15:12:01.890356: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 15:12:01.890399: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 15:12:01.890438: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 15:12:01.897826: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.




mlp sentiment accuracy:  0.834


/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)




mlp PERCY score:  0.025


cnn sentiment accuracy:  0.892


/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


...



gru PERCY score:  0.065


bigru sentiment accuracy:  0.872


/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)




bigru PERCY score:  0.07


transformer sentiment accuracy:  0.889


/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)




transformer PERCY score:  0.059
